# PCR Primer design
So far we have used Snapgene to design our primers. However in order to explore open-source alternatives this notebook will use `Biopython` in conjunction with Primer3
## Installation
First we need to install the python wrapper `primer3-py`

In [2]:
!pip install primer3-py

## Import
We can now import it along with other libraries we need

In [3]:
from Bio import SeqIO
import primer3
import os
os.chdir("..")

## Designing our first primer

In [4]:
dna_seq = str("AGCGCACATCTCTAATCTTTGATTTACGTATAATCGTTACACAACCATCCCCCTAATTCCTCTACAGAGGCGCGTCCTGCTGCCCCTTCCCATGGCACATGCTTCCAACTTACGATTTCTCACGTCCCACTCGTCCCAGGTTTCGCCTTTTCGGATTTCGTCCGTGTTGTGTTTTGTCTTCTTCCGTTGTCTTCCTCCGTTGTCTTCGATTTGATTCCCCACACCCGACTGCCTATCATTGTTCTTGCTTCGGATTCCTGGGCCACCTCGTTGAACTGGTGCTACTCAATAATTTATCGTAATTTACCATACGTTATCGATACATACTCCCCCCCATTCCGTTGTATTCCTTCGCTGTAGCTTTTGAGACCACCACCACCACCCTAGCCTCTGTTGTATAGTTTTCGACTTCGTCCGCCCGCCGTTGTCCTTGTGAACCATTTTTGTTGCTATCGCACACATACATACCAAGTACACTGCTTCTTGCTATCCTCCCTACTAGCTTTCTACATGCATTGCTACTATACAGCCCTCTACAGCACTTGAAATGAAGTAATTTGAGCCCGGAGCATCCACGAAACCTACGCGAGGAAGAGGTTCTTTAGGCGCTGGTCTGCATTTTCGCCCATACTCAGTCCCAAATTCCAGGGAACGAGGCCAGTAATACTACTGTATTGATTGTCGTTTTACAGCCACTGTCACTGTCATTCAATGTCAACGAACGGCGTCCAGACTTGTCAAGCATCGTGCCAACGATTTGTTTCCGCGTTCACAGCATCCGCTCCAACATCTCCCTATTTTCGACGTACAAATCAAACCCTAGTTGACAGCAAACCTAACGGTAGTGTTGTTGCGAGGTGGAGGTCCCTCAAGGTGGAGTTACCTTGAGGTCCCGTATTTGTGATGGTGGCGTTCTCAAGGTCCCTTGAGGTGGCCTTGATGTGTTTTCCCATCCCTGTAACGGAATCTCCGGGACGGTGATGTGGCAGCGAGTGCGAGC")

primers = primer3.bindings.design_primers({'SEQUENCE_TEMPLATE': dna_seq,}, {'PRIMER_OPT_SIZE': 20,'PRIMER_MIN_SIZE': 18, 'PRIMER_MAX_SIZE': 25, 'PRIMER_OPT_TM': 60.0, 'PRIMER_MIN_TM': 57.0, 'PRIMER_MAX_TM': 63.0, 'PRIMER_PAIR_MAX_DIFF_TM': 3.0, 'PRIMER_PRODUCT_SIZE_RANGE': [[900, 1100]],'SEQUENCE_INCLUDED_REGION': [0, len(dna_seq)],'SEQUENCE_PRIMER_PAIR_OK_REGION_LIST': [[0, 25, len(dna_seq)-25, 25]]})

# Print the first primer pair
print("Forward primer:", primers['PRIMER_LEFT_0_SEQUENCE'])
print("Reverse primer:", primers['PRIMER_RIGHT_0_SEQUENCE'])
print("Product size:", primers['PRIMER_PAIR_0_PRODUCT_SIZE'])

Forward primer: AGCGCACATCTCTAATCTTTGA
Reverse primer: ACTCGCTGCCACATCACC
Product size: 994


As we can see the primer is slightly shorter than the sequence due to primer3's optimization even when specifying strict positions with the last argument. In this case we choose to ignore it as it will likely not be a problem however if an important motif is left out the promoter will not work very well. To solve this one could screen the input sequence for restriction sites and choose a non-cutter restriction sequence to append to the primer and include a wider piece of the genome to ensure the entire upstream region is extracted.
## Designing primers for genomic extraction of upstream sequences

In [5]:
pth = "data\primer_design\promoters_for_extraction.fasta"

def design_primers(pth):
    for record in SeqIO.parse(pth, "fasta"):
        dna_seq = str(record.seq)
        primers = primer3.bindings.design_primers({'SEQUENCE_TEMPLATE': dna_seq,}, {'PRIMER_OPT_SIZE': 20,'PRIMER_MIN_SIZE': 18, 'PRIMER_MAX_SIZE': 25, 'PRIMER_OPT_TM': 60.0, 'PRIMER_MIN_TM': 57.0, 'PRIMER_MAX_TM': 63.0, 'PRIMER_PAIR_MAX_DIFF_TM': 3.0, 'PRIMER_PRODUCT_SIZE_RANGE': [[900, 1100]],'SEQUENCE_INCLUDED_REGION': [0, len(dna_seq)],'SEQUENCE_PRIMER_PAIR_OK_REGION_LIST': [[0, 50, len(dna_seq)-50, 50]]})
        print(f"Record: {record.id}")
        print("up primer:", primers['PRIMER_LEFT_0_SEQUENCE'])
        print("down primer:", primers['PRIMER_RIGHT_0_SEQUENCE'])
        print("product size:", primers['PRIMER_PAIR_0_PRODUCT_SIZE'])
        print("primr melting temp: ", primers['PRIMER_LEFT_0_TM'], primers['PRIMER_RIGHT_0_TM'])
        print()

In [6]:
from src.utils import design_primers

design_primers(pth)

Record: PKG1_promoter
up primer: ATCTGGTCGTGATTCGCCAC
down primer: TCGTGAAGGAACGGAATGGT
product size: 971
primr melting temp:  60.46028408477713 59.317148410654

Record: ADH1_promoter
up primer: CAGGGTCAAGCAGAGCAGAA
down primer: GGTTGATGCTGAGTGTGATCG
product size: 989
primr melting temp:  59.96411926156776 59.33737138101719

Record: TDH3_promoter
up primer: CGTTCCCACTTTGGACGTGT
down primer: TGTGGTGGATTGGGTGATGA
product size: 975
primr melting temp:  60.812412072024244 58.928565557755405

Record: ACT_promoter
up primer: TCGAGAAGAGAGGTAGGCGG
down primer: AAGGAAGAAAACTGGTGGTTGA
product size: 969
primr melting temp:  60.464013016904346 58.02398523353452

Record: TEF1_promoter
up primer: TTATGTTCTCCTCGTGGGCG
down primer: ATCGTCAGCATCCATCTCGC
product size: 953
primr melting temp:  59.825155079357785 60.31938261960141

Record: DED1_promoter
up primer: CCACAGATGCAAACGCAACA
down primer: TCGTTCTTGAATCGCTGTGGA
product size: 943
primr melting temp:  59.96949614280919 59.99960552843305

Record: gpd